## Import & Environment Setting

In [1]:
import torch
from torch import autograd
from torch.utils.data import DataLoader

import json
import gc
import numpy as np

from utils.dataset import LabeledDataset, VideoDataset
from utils.model import YoloV3, YoloLoss
from utils.postprocess import PostProcessor

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

import os
import fnmatch
from collections import Counter

import urllib
from io import BytesIO

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float

## Load Config

In [3]:
with open("./config/config.json", "r") as config_file:
    main_config = json.load(config_file)

try:
    model_config = main_config['model']
    test_config = main_config['test']
except NameError:
    assert False, ('Failed to load config file')
except KeyError:
    assert False, ('Failed to find key on config file')

In [4]:
model_config['device'] = device
model_config['dtype'] = dtype
model_config['attrib_count'] = 5 + model_config['class_count']

test_config['device'] = device
test_config['dtype'] = dtype

## Build

In [5]:
model = torch.load('./model/run3/model_r_143.dat')
model.to(device)
model = model.eval()

In [6]:
test_context = { }

test_context['dataset'] = LabeledDataset(test_config['test_list'], test_config['test_image'], test_config['test_label'])
test_context['dataloader'] = DataLoader(test_context['dataset'], batch_size = 1, num_workers = 1)

test_context['post_conf_threshold'] = test_config['post_conf_threshold']
test_context['post_iou_threshold'] = test_config['post_iou_threshold']
test_context['acc_iou_threshold'] = test_config['acc_iou_threshold']

test_context['video_dir'] = test_config['video_dir']
test_context['font'] = ImageFont.truetype("arial.ttf", 16)

test_context['debug_level'] = test_config['debug_level']

In [7]:
def iou(bbox1, bbox2):
    w1 = bbox1[2]
    h1 = bbox1[3]
    
    w2 = bbox1[2]
    h2 = bbox1[3]
    
    left1 = bbox1[0] - w1 / 2
    left2 = bbox2[0] - w2 / 2
    right1 = bbox1[0] + w1 / 2
    right2 = bbox2[0] + w2 / 2
    top1 = bbox1[1] + h1 / 2
    top2 = bbox2[1] + h2 / 2
    bottom1 = bbox1[1] - h1 / 2
    bottom2 = bbox2[1] - h2 / 2
    
    area1 = w1 * h1
    area2 = w2 * h2
    
    w_intersect = min(right1, right2) - max(left1, left2)
    h_intersect = min(top1, top2) - max(bottom1, bottom2)
    area_intersect = h_intersect * w_intersect
    
    if w_intersect < 0 or h_intersect < 0:
        return 0

    iou_ = area_intersect / (area1 + area2 - area_intersect)
    return iou_
        
def plot_pred(test_context, prediction, plot_options = ['ABOVE'], print_accuracy = False):
        
    if test_context['debug_level'] >= 1:
        print('title : ', prediction['title'])
        print('plot_options : ', plot_options)
        
    pred = prediction['pred']
    image = prediction['image']
    label = prediction['label']
    label_len = prediction['label_len']
    
    postProcessor = PostProcessor()
    
    ax_indx = 0
    _, ax = plt.subplots(1, len(plot_options), figsize=(8 * len(plot_options), 8))
    
    for plot_opt in plot_options:
        # plot all boundingbox above threshold
        if plot_opt is 'ABOVE':
            pred_result = postProcessor.ABOVE(pred, test_context)
            accuracy = None
        elif plot_opt is 'NMS':
            pred_result = postProcessor.NMS(pred, test_context)
            if print_accuracy is True:
                accuracy = postProcessor.calcAccuracyMap(label, label_len, pred_result, test_context)
                print('NMS accuracy : ', accuracy)
        elif plot_opt is 'CUSTOM1':
            pred_result = postProcessor.CUSTOM1(pred, test_context)
            if print_accuracy is True:
                accuracy = postProcessor.calcAccuracyMap(label, label_len, pred_result, test_context)
                print('CUSTOM1 accuracy : ', accuracy)
        elif plot_opt is 'CUSTOM2':
            pred_result = postProcessor.CUSTOM2(pred, test_context)
            if print_accuracy is True:
                accuracy = postProcessor.calcAccuracyMap(label, label_len, pred_result, test_context)
                print('CUSTOM2 accuracy : ', accuracy)
        else:
            ax_indx = ax_indx + 1
            continue
            
            
        if len(plot_options) > 1:
            ax[ax_indx].imshow(image)
        else:
            ax.imshow(image)
                
        for bbox in pred_result:
            color = '%02x'% int(255 * (bbox[4] - test_context['post_conf_threshold']) / (1 - test_context['post_conf_threshold']))
            color = '#' + str(color) + str('00') + str('00')
            
            bounding = patches.Rectangle((bbox[0] - bbox[2] / 2, bbox[1] - bbox[3] / 2), bbox[2], bbox[3], 
                        linewidth=1, edgecolor=color, facecolor='none')
                
            if len(plot_options) > 1:
                ax[ax_indx].add_patch(bounding)
            else:
                ax.add_patch(bounding)
        ax_indx = ax_indx + 1
    
        
    plt.show()
    plt.close('all')
        
    gc.collect()
    torch.cuda.empty_cache()

    
def plot_save(test_context, target_name, prediction, size = (1920, 1080), plot_opt = 'CUSTOM2'):
        
    if test_context['debug_level'] >= 1:
        print('title : ', prediction['title'])
        
    postProcessor = PostProcessor()
    prediction = postProcessor.resize(from_size = (608, 608), to_size = size, prediction = prediction)
    
    pred = prediction['pred']
    image = prediction['image']
    label = prediction['label']
    
    if plot_opt is 'ABOVE':
        pred_result = postProcessor.ABOVE(pred, test_context)
    elif plot_opt is 'NMS':
        pred_result = postProcessor.NMS(pred, test_context)
    elif plot_opt is 'CUSTOM1':
        pred_result = postProcessor.CUSTOM1(pred, test_context)
    elif plot_opt is 'CUSTOM2':
        pred_result = postProcessor.CUSTOM2(pred, test_context)
    else:
        return
    
    imagepath = test_context['video_dir'] + target_name + '/' + prediction['title']
    image = Image.open(imagepath)
    image_pixels = image.load()
    
    draw = ImageDraw.Draw(image)

    print(len(pred_result), 'boxes found')
    for bbox in pred_result:
        left = int(bbox[0] - bbox[2] / 2)
        right = int(bbox[0] + bbox[2] / 2)
        up = int(bbox[1] - bbox[3] / 2)
        down = int(bbox[1] + bbox[3] / 2)
        
        if left >= 0 and up >= 0:
            draw.text((left, up), str(bbox[4]),(255,0,0),font=test_context['font'])
        elif left >= 0 and up < 0:
            draw.text((left, down), str(bbox[4]),(255,0,0),font=test_context['font'])
        elif left < 0 and up >= 0:
            draw.text((right, up), str(bbox[4]),(255,0,0),font=test_context['font'])
        else:
            draw.text((right, up), str(bbox[4]),(255,0,0),font=test_context['font'])
            
            
        draw.rectangle(xy=[left, up, right, down], outline=(255, 0, 0) )
    output_dir = test_context['video_dir'] + 'o_' + target_name + '/'
    image.save(output_dir + prediction['title'][:-3] + 'png')
    
    gc.collect()
    torch.cuda.empty_cache()
    
        
def test_all(test_context, test_config, plot_options = ['ABOVE', 'NMS', 'CUSTOM2']):
    accs = Counter({})
    
    with torch.no_grad():
        for idx, batches in enumerate(test_context['dataloader']):
            image = batches['image'].to(test_config['device'], dtype = test_config['dtype'])
            labels = batches['label'].to(test_config['device'], dtype = test_config['dtype'])
            label_len = batches['label_len'].to(test_config['device'], dtype = torch.long)
            
            out1, out2, out3 = model(image)
            pred = torch.cat((out1, out2, out3), 1)
        
            output = {}
            output['title'] = batches['title']
            output['image'] = batches['image'].cpu().permute(0, 2, 3, 1).squeeze(0).numpy()
            output['pred'] = pred.cpu().detach().squeeze(0).numpy()
            output['label'] = batches['label'].cpu().squeeze(0).numpy()
            output['label_len'] = batches['label_len'].cpu().squeeze(0).numpy()
            
            plot_pred(test_context, output, plot_options, print_accuracy = True)
        
            postProcessor = PostProcessor()
            pred_result = postProcessor.CUSTOM2(output['pred'], test_context)
            accuracy = postProcessor.calcAccuracyMap(output['label'], output['label_len'], pred_result, test_context)
            accs = accs + Counter(accuracy)
            
            del image, labels, label_len
            del out1, out2, out3, output
            gc.collect()
            torch.cuda.empty_cache()
        print('total : ', accs)
        tp = accs['true positive']
        fn = accs['false negative']
        fp = accs['false positive'] + accs['duplicate']
        print('accuracy : ', tp / (tp + fn + fp))

def test_n(test_context, test_config, n = 1, plot_options = ['ABOVE', 'NMS', 'CUSTOM2']):
    with torch.no_grad():
        for idx, batches in enumerate(test_context['dataloader']):
            image = batches['image'].to(test_config['device'], dtype = test_config['dtype'])
            labels = batches['label'].to(test_config['device'], dtype = test_config['dtype'])
            label_len = batches['label_len'].to(test_config['device'], dtype = torch.long)
            
            out1, out2, out3 = model(image)
            pred = torch.cat((out1, out2, out3), 1)
        
            output = {}
            output['title'] = batches['title']
            output['image'] = batches['image'].cpu().permute(0, 2, 3, 1).squeeze(0).numpy()
            output['pred'] = pred.cpu().detach().squeeze(0).numpy()
            output['label'] = batches['label'].cpu().squeeze(0).numpy()
            output['label_len'] = batches['label_len'].cpu().squeeze(0).numpy()
            
            plot_pred(test_context, output, plot_options, print_accuracy = True)
        
            del image, labels, label_len
            del out1, out2, out3, output
            gc.collect()
            torch.cuda.empty_cache()
            
            if idx + 1 >= n:
                return
            
def test_url_image(test_context, test_config, url, crop = False, crop_size = (0, 0, 1920, 1080), 
                   plot_options = ['ABOVE', 'NMS', 'CUSTOM2']):
    image = Image.open(urllib.request.urlopen(url))
    if crop is True:
        image = image.crop(crop_size)
    image = np.array(image.resize((608, 608), Image.BILINEAR))[:, :, :3]
    image = torch.from_numpy(image).unsqueeze(0).float() / 255
    image = image.permute(0, 3, 1, 2)
    
    image = image.to(test_config['device'], dtype = test_config['dtype'])
        
    out1, out2, out3 = model(image)
        
    pred = torch.cat((out1, out2, out3), 1)
        
    output = {}
    output['title'] = url if len(url) < 80 else url[0:77] + '...' 
    output['image'] = image.permute(0, 2, 3, 1).cpu().squeeze(0).numpy()
    output['pred'] = pred.cpu().detach().squeeze(0).numpy()
    output['label'] = []
    output['label_len'] = 0
        
    plot_pred(test_context, output, plot_options)
    
    gc.collect()
    torch.cuda.empty_cache()
    
def test_video(test_context, target_name, start = 0, size=(1920, 1080), plot_opt = 'CUSTOM2'):
    file_list = os.listdir(test_context['video_dir'] + target_name + '/')
    file_list = np.sort(file_list)
    file_list = file_list[start:]
    for image_name in file_list:
        if fnmatch.fnmatch(image_name, '*.png') is False:
            continue
        print(image_name)
        min_rate = min(608 / size[0], 608 / size[1])
        orig_image = Image.open(test_context['video_dir'] + target_name + '/' + image_name)
        orig_image = orig_image.resize((int(size[0] * min_rate), int(size[1] * min_rate)), Image.NEAREST)
        image = Image.new("RGB", (608, 608))
        image.paste(orig_image, (0, 0, int(size[0] * min_rate), int(size[1] * min_rate)))
        image = np.array(image)[:, :, :3]
        image = torch.from_numpy(image).unsqueeze(0).float() / 255
        image = image.permute(0, 3, 1, 2)
    
        image = image.to(test_config['device'], dtype = test_config['dtype'])
        
        out1, out2, out3 = model(image)
        
        pred = torch.cat((out1, out2, out3), 1)
        
        output = {}
        output['title'] = image_name
        output['image'] = image.permute(0, 2, 3, 1).cpu().squeeze(0).numpy()
        output['pred'] = pred.cpu().detach().squeeze(0).numpy()
        output['label'] = []
        output['label_len'] = 0
        
        plot_save(test_context, target_name, output, size = size, plot_opt = plot_opt)
            
        del image
        del out1, out2, out3
        del output
        gc.collect()
        torch.cuda.empty_cache()

In [8]:
#test_all(test_context, test_config)

In [9]:
#with open("../data/detect/links/url_links.txt", "r") as url_files:
#    lines = url_files.readlines()
#    for url in lines:
#        try:
#            test_context['debug_level'] = 1
#            test_url_image(test_context, test_config, url)
#        except urllib.error.HTTPError:
#            print('http error')

In [10]:
#test_video(test_context, 'toaru_majutsu_op', start = 0, size=(1280, 720), plot_opt = 'CUSTOM2')

In [11]:
def test_video2(context, config, target_name, size=(1920, 1080), splits = (1, 1), batch_multiplier = 1, plot_opt = 'CUSTOM2'):
    split_count = splits[0] * splits[1]
    batch_size = batch_multiplier * split_count
    
    image_dir = test_context['video_dir'] + target_name + '/'
    dataset = VideoDataset(image_dir, splits = splits, from_size = size, to_size = (608, 608))
    loader = DataLoader(dataset, batch_size = batch_size, num_workers = 1)
    postprocess = PostProcessor()
    
    with torch.no_grad():
        for idx, batches in enumerate(loader):
            image = batches['image'].to(config['device'], dtype = config['dtype'])
            
            out1, out2, out3 = model(image)
            pred = torch.cat((out1, out2, out3), 1)
            
            for i in range(0, batches['image'].shape[0] // split_count):
                # processing image
                print(batches['title'][i * split_count])
                
                # messy algorithm on gpu
                # 1.recover prediction coordinate
                for j in range(0, split_count):
                    crop_size = batches['crop_size'][i * split_count + j]
                    crop_bbox = batches['crop_bbox'][i * split_count + j]
                    
                    crop_mult = torch.as_tensor([float(crop_size[0]) / 608, 
                                          float(crop_size[1]) / 608, 
                                          float(crop_size[0]) / 608, 
                                          float(crop_size[1]) / 608.0]).to(config['device'], dtype = config['dtype'])
                    crop_add = torch.as_tensor([crop_bbox[0], 
                                                crop_bbox[1], 
                                                0, 
                                                0]).to(config['device'], dtype = config['dtype'])
                    
                    pred[i * split_count + j][:, :4].mul_(crop_mult).add_(crop_add)
                
                # 2.concat splits
                pred_result = torch.cat(tuple(pred[i * split_count : (i + 1) * split_count]), dim = 0).cpu().detach()
                pred_result = np.copy(pred_result[np.where(pred_result[:, 4] > context['post_conf_threshold'])])
                
                # run NMS / CUSTOM2
                
                print(pred_result.shape)
                pred_result = postprocess.CUSTOM2(pred_result, context)
                print(len(pred_result), 'boxes found')
            
                
                # to pillow
                image_to_save = batches['image_og'][i * split_count].cpu().numpy()
                image_to_save = Image.fromarray(image_to_save.astype('uint8'), 'RGB')
            
                # draw image
                draw = ImageDraw.Draw(image_to_save)
                for bbox in pred_result:
                    left = int(bbox[0] - bbox[2] / 2)
                    right = int(bbox[0] + bbox[2] / 2)
                    up = int(bbox[1] - bbox[3] / 2)
                    down = int(bbox[1] + bbox[3] / 2)
        
                    draw.text((left, down), str(bbox[4]),(255,0,0),font=test_context['font'])
                    draw.rectangle(xy=[left, up, right, down], outline=(255, 0, 0) )
            
                # save image
                output_dir = test_context['video_dir'] + 'o_' + target_name + '/'
                image_to_save.save(output_dir + batches['title'][i * split_count][:-3] + 'png')
                
                # show image
                #_, ax = plt.subplots(figsize=(16, 9))
                #ax.imshow(np.array(image_to_save))
                #plt.show()
            
            del image, batches
            del out1, out2, out3, pred
            gc.collect()
            torch.cuda.empty_cache()
            

In [ ]:
test_video2(test_context, test_config, 'nozaki_op', size=(1280, 720), splits = (2, 1), batch_multiplier = 16, plot_opt = 'CUSTOM2')

nozaki_op0001.png
(0, 5)
0 boxes found
nozaki_op0002.png
(0, 5)
0 boxes found
nozaki_op0003.png
(0, 5)
0 boxes found
nozaki_op0004.png
(0, 5)
0 boxes found
nozaki_op0005.png
(0, 5)
0 boxes found
nozaki_op0006.png
(0, 5)
0 boxes found
nozaki_op0007.png
(0, 5)
0 boxes found
nozaki_op0008.png
(0, 5)
0 boxes found
nozaki_op0009.png
(0, 5)
0 boxes found
nozaki_op0010.png
(0, 5)
0 boxes found
nozaki_op0011.png
(0, 5)
0 boxes found
nozaki_op0012.png
(0, 5)
0 boxes found
nozaki_op0013.png
(0, 5)
0 boxes found
nozaki_op0014.png
(0, 5)
0 boxes found
nozaki_op0015.png
(0, 5)
0 boxes found
nozaki_op0016.png
(0, 5)
0 boxes found
nozaki_op0017.png
(0, 5)
0 boxes found
nozaki_op0018.png
(0, 5)
0 boxes found
nozaki_op0019.png
(0, 5)
0 boxes found
nozaki_op0020.png
(0, 5)
0 boxes found
nozaki_op0021.png
(0, 5)
0 boxes found
nozaki_op0022.png
(0, 5)
0 boxes found
nozaki_op0023.png
(0, 5)
0 boxes found
nozaki_op0024.png
(0, 5)
0 boxes found
nozaki_op0025.png
(0, 5)
0 boxes found
nozaki_op0026.png
(0, 5)


0 boxes found
nozaki_op0210.png
(0, 5)
0 boxes found
nozaki_op0211.png
(0, 5)
0 boxes found
nozaki_op0212.png
(0, 5)
0 boxes found
nozaki_op0213.png
(0, 5)
0 boxes found
nozaki_op0214.png
(0, 5)
0 boxes found
nozaki_op0215.png
(0, 5)
0 boxes found
nozaki_op0216.png
(0, 5)
0 boxes found
nozaki_op0217.png
(0, 5)
0 boxes found
nozaki_op0218.png
(0, 5)
0 boxes found
nozaki_op0219.png
(0, 5)
0 boxes found
nozaki_op0220.png
(0, 5)
0 boxes found
nozaki_op0221.png
(0, 5)
0 boxes found
nozaki_op0222.png
(0, 5)
0 boxes found
nozaki_op0223.png
(0, 5)
0 boxes found
nozaki_op0224.png
(0, 5)
0 boxes found
nozaki_op0225.png
(0, 5)
0 boxes found
nozaki_op0226.png
(0, 5)
0 boxes found
nozaki_op0227.png
(0, 5)
0 boxes found
nozaki_op0228.png
(0, 5)
0 boxes found
nozaki_op0229.png
(0, 5)
0 boxes found
nozaki_op0230.png
(0, 5)
0 boxes found
nozaki_op0231.png
(0, 5)
0 boxes found
nozaki_op0232.png
(0, 5)
0 boxes found
nozaki_op0233.png
(0, 5)
0 boxes found
nozaki_op0234.png
(0, 5)
0 boxes found
nozaki_op02

nozaki_op0383.png
(15, 5)
1 boxes found
[968 134 120 113   0]
nozaki_op0384.png
(22, 5)
1 boxes found
[976 109 135 126   0]
nozaki_op0385.png
(22, 5)
1 boxes found
[974 110 136 127   0]
nozaki_op0386.png
(22, 5)
1 boxes found
[974 111 136 127   0]
nozaki_op0387.png
(31, 5)
1 boxes found
[981 113 113 108   0]
nozaki_op0388.png
(31, 5)
1 boxes found
[990 115 110 105   0]
nozaki_op0389.png
(32, 5)
1 boxes found
[974 113 114 108   0]
nozaki_op0390.png
(35, 5)
1 boxes found
[970 119 105 100   0]
nozaki_op0391.png
(35, 5)
1 boxes found
[968 122 105 100   0]
nozaki_op0392.png
(21, 5)
1 boxes found
[972 117 137 129   0]
nozaki_op0393.png
(21, 5)
1 boxes found
[973 120 137 129   0]
nozaki_op0394.png
(21, 5)
1 boxes found
[970 121 137 129   0]
nozaki_op0395.png
(23, 5)
1 boxes found
[978 117 139 131   0]
nozaki_op0396.png
(23, 5)
1 boxes found
[975 117 138 130   0]
nozaki_op0397.png
(23, 5)
1 boxes found
[978 118 139 131   0]
nozaki_op0398.png
(39, 5)
1 boxes found
[988 114 102  98   0]
nozaki_o

[748 255 375 348   0]
nozaki_op0515.png
(44, 5)
1 boxes found
[749 227 358 335   0]
nozaki_op0516.png
(46, 5)
1 boxes found
[739 227 351 332   0]
nozaki_op0517.png
(43, 5)
1 boxes found
[719 228 366 342   0]
nozaki_op0518.png
(25, 5)
1 boxes found
[760 210 351 334   0]
nozaki_op0519.png
(43, 5)
1 boxes found
[744 232 350 327   0]
nozaki_op0520.png
(34, 5)
1 boxes found
[742 240 345 327   0]
nozaki_op0521.png
(0, 5)
0 boxes found
nozaki_op0522.png
(0, 5)
0 boxes found
nozaki_op0523.png
(0, 5)
0 boxes found
nozaki_op0524.png
(0, 5)
0 boxes found
nozaki_op0525.png
(0, 5)
0 boxes found
nozaki_op0526.png
(0, 5)
0 boxes found
nozaki_op0527.png
(14, 5)
1 boxes found
[1060  141  132  124    0]
nozaki_op0528.png
(14, 5)
1 boxes found
[1060  141  132  124    0]
nozaki_op0529.png
(14, 5)
1 boxes found
[1061  141  132  124    0]
nozaki_op0530.png
(12, 5)
1 boxes found
[1070  140  135  127    0]
nozaki_op0531.png
(13, 5)
1 boxes found
[1065  140  134  126    0]
nozaki_op0532.png
(12, 5)
1 boxes fou

0 boxes found
nozaki_op0637.png
(0, 5)
0 boxes found
nozaki_op0638.png
(0, 5)
0 boxes found
nozaki_op0639.png
(0, 5)
0 boxes found
nozaki_op0640.png
(0, 5)
0 boxes found
nozaki_op0641.png
(0, 5)
0 boxes found
nozaki_op0642.png
(0, 5)
0 boxes found
nozaki_op0643.png
(0, 5)
0 boxes found
nozaki_op0644.png
(31, 5)
1 boxes found
[980 141 230 210   0]
nozaki_op0645.png
(0, 5)
0 boxes found
nozaki_op0646.png
(25, 5)
1 boxes found
[385 217 256 235   0]
nozaki_op0647.png
(7, 5)
2 boxes found
[526 101  54  53   0]
[392 199 274 260   0]
nozaki_op0648.png
(23, 5)
2 boxes found
[402 208 274 260   0]
[527 100  54  54   0]
nozaki_op0649.png
(0, 5)
0 boxes found
nozaki_op0650.png
(0, 5)
0 boxes found
nozaki_op0651.png
(3, 5)
1 boxes found
[ 82 388  59  57   0]
nozaki_op0652.png
(0, 5)
0 boxes found
nozaki_op0653.png
(4, 5)
1 boxes found
[401 236 269 250   0]
nozaki_op0654.png
(0, 5)
0 boxes found
nozaki_op0655.png
(0, 5)
0 boxes found
nozaki_op0656.png
(1, 5)
1 boxes found
[145 391  54  54   0]
nozak

...

1 boxes found
[134 530  55  54   0]
nozaki_op1260.png
(1, 5)
1 boxes found
[134 530  55  54   0]
nozaki_op1261.png
(0, 5)
0 boxes found
nozaki_op1262.png
(0, 5)
0 boxes found
nozaki_op1263.png
(1, 5)
1 boxes found
[134 530  55  54   0]
nozaki_op1264.png
(1, 5)
1 boxes found
[134 530  55  54   0]
nozaki_op1265.png
(1, 5)
1 boxes found
[135 530  55  54   0]
nozaki_op1266.png
(2, 5)
1 boxes found
[135 530  58  54   0]
nozaki_op1267.png
(2, 5)
1 boxes found
[135 530  55  54   0]
nozaki_op1268.png
(2, 5)
1 boxes found
[135 530  55  54   0]
nozaki_op1269.png
(2, 5)
1 boxes found
[135 530  61  56   0]
nozaki_op1270.png
(2, 5)
1 boxes found
[135 530  55  54   0]
nozaki_op1271.png
(2, 5)
1 boxes found
[135 530  58  54   0]
nozaki_op1272.png
(2, 5)
1 boxes found
[135 530  55  54   0]
nozaki_op1273.png
(2, 5)
1 boxes found
[135 530  55  54   0]
nozaki_op1274.png
(1, 5)
1 boxes found
[136 530  55  54   0]
nozaki_op1275.png
(1, 5)
1 boxes found
[135 530  55  54   0]
nozaki_op1276.png
(2, 5)
1 boxes

0 boxes found
nozaki_op1396.png
(0, 5)
0 boxes found
nozaki_op1397.png
(0, 5)
0 boxes found
nozaki_op1398.png
(0, 5)
0 boxes found
nozaki_op1399.png
(0, 5)
0 boxes found
nozaki_op1400.png
(0, 5)
0 boxes found
nozaki_op1401.png
(0, 5)
0 boxes found
nozaki_op1402.png
(0, 5)
0 boxes found
nozaki_op1403.png
(0, 5)
0 boxes found
nozaki_op1404.png
(0, 5)
0 boxes found
nozaki_op1405.png
(0, 5)
0 boxes found
nozaki_op1406.png
(0, 5)
0 boxes found
nozaki_op1407.png
(6, 5)
1 boxes found
[1020  192  537  494    0]
nozaki_op1408.png
(11, 5)
1 boxes found
[1013  199  540  493    0]
nozaki_op1409.png
(1, 5)
1 boxes found
[1031  196  568  523    0]
nozaki_op1410.png
(0, 5)
0 boxes found
nozaki_op1411.png
(0, 5)
0 boxes found
nozaki_op1412.png
(0, 5)
0 boxes found
nozaki_op1413.png
(0, 5)
0 boxes found
nozaki_op1414.png
(0, 5)
0 boxes found
nozaki_op1415.png
(0, 5)
0 boxes found
nozaki_op1416.png
(0, 5)
0 boxes found
nozaki_op1417.png
(0, 5)
0 boxes found
nozaki_op1418.png
(0, 5)
0 boxes found
nozaki_

nozaki_op1599.png
(15, 5)
2 boxes found
[46 79 85 83  0]
[183  75  88  85   0]
nozaki_op1600.png
(14, 5)
2 boxes found
[51 89 91 88  0]
[184  75  67  60   0]
nozaki_op1601.png
(7, 5)
1 boxes found
[52 88 80 76  0]
nozaki_op1602.png
(2, 5)
1 boxes found
[56 83 86 83  0]
nozaki_op1603.png
(0, 5)
0 boxes found
nozaki_op1604.png
(0, 5)
0 boxes found
nozaki_op1605.png
(0, 5)
0 boxes found
nozaki_op1606.png
(0, 5)
0 boxes found
nozaki_op1607.png
(14, 5)
1 boxes found
[59 89 79 71  0]
nozaki_op1608.png
(15, 5)
1 boxes found
[58 90 74 69  0]
nozaki_op1609.png
(20, 5)
1 boxes found
[65 77 85 82  0]
nozaki_op1610.png
(20, 5)
1 boxes found
[77 85 90 87  0]
nozaki_op1611.png
(16, 5)
1 boxes found
[66 85 76 74  0]
nozaki_op1612.png
(31, 5)
1 boxes found
[68 94 94 91  0]
nozaki_op1613.png
(26, 5)
1 boxes found
[67 78 92 88  0]
nozaki_op1614.png
(23, 5)
1 boxes found
[69 81 89 86  0]
nozaki_op1615.png
(23, 5)
1 boxes found
[74 93 88 84  0]
nozaki_op1616.png
(15, 5)
1 boxes found
[ 69 100  64  61   0]

nozaki_op1790.png
(0, 5)
0 boxes found
nozaki_op1791.png
(0, 5)
0 boxes found
nozaki_op1792.png
(54, 5)
1 boxes found
[1138  288  392  358    0]
nozaki_op1793.png
(60, 5)
1 boxes found
[1124  292  381  357    0]
nozaki_op1794.png
(104, 5)
1 boxes found
[1046  285  464  415    0]
nozaki_op1795.png
(101, 5)
1 boxes found
[1018  268  498  465    0]
nozaki_op1796.png
(97, 5)
1 boxes found
[984 321 494 423   0]
nozaki_op1797.png
(96, 5)
1 boxes found
[1023  321  487  414    0]
nozaki_op1798.png
(61, 5)
1 boxes found
[962 286 505 405   0]
nozaki_op1799.png
(69, 5)
1 boxes found
[903 284 399 365   0]
nozaki_op1800.png
(81, 5)
1 boxes found
[878 281 445 404   0]
nozaki_op1801.png
(76, 5)
1 boxes found
[871 280 441 411   0]
nozaki_op1802.png
(78, 5)
1 boxes found
[816 262 447 382   0]
nozaki_op1803.png
(78, 5)
1 boxes found
[817 260 439 378   0]
nozaki_op1804.png
(73, 5)
1 boxes found
[806 249 465 364   0]
nozaki_op1805.png
(73, 5)
1 boxes found
[822 254 459 371   0]
nozaki_op1806.png
(77, 5)
1

(0, 5)
0 boxes found
nozaki_op1945.png
(0, 5)
0 boxes found
nozaki_op1946.png
(0, 5)
0 boxes found
nozaki_op1947.png
(0, 5)
0 boxes found
nozaki_op1948.png
(0, 5)
0 boxes found
nozaki_op1949.png
(0, 5)
0 boxes found
nozaki_op1950.png
(0, 5)
0 boxes found
nozaki_op1951.png
(0, 5)
0 boxes found
nozaki_op1952.png
(0, 5)
0 boxes found
nozaki_op1953.png
(0, 5)
0 boxes found
nozaki_op1954.png
(0, 5)
0 boxes found
nozaki_op1955.png
(0, 5)
0 boxes found
nozaki_op1956.png
(0, 5)
0 boxes found
nozaki_op1957.png
(0, 5)
0 boxes found
nozaki_op1958.png
(0, 5)
0 boxes found
nozaki_op1959.png
(0, 5)
0 boxes found
nozaki_op1960.png
(0, 5)
0 boxes found
nozaki_op1961.png
(0, 5)
0 boxes found
nozaki_op1962.png
(0, 5)
0 boxes found
nozaki_op1963.png
(0, 5)
0 boxes found
nozaki_op1964.png
(0, 5)
0 boxes found
nozaki_op1965.png
(0, 5)
0 boxes found
nozaki_op1966.png
(0, 5)
0 boxes found
nozaki_op1967.png
(0, 5)
0 boxes found
nozaki_op1968.png
(0, 5)
0 boxes found
nozaki_op1969.png
(0, 5)
0 boxes found
noza